#### Abreviaturas:
- df: data frame
- cg_coef: coeficientes de Clebsch Gordan para SU(3)
- dtot: número de degeneraciones totales

In [16]:
import pandas as pd
import numpy as np
import math

In [17]:
# Se importa el archivo
df = pd.read_csv('8x8comp.txt', skiprows=4, header=None)

# Se agregan las cabeceras de las columnas
df.columns = ['k', 'l', 'm', 'k1', 'l1', 'm1', 'k2', 'l2', 'm2', 'cg_coef', 'p', 'q', 'degeneracy', 'dtot']

# Se eliminan las filas con valores NaN
df = df.dropna()

# Convierte todas las columnas a tipo int con excepción de "cg_doef"
df = df.astype({"degeneracy":int, "dtot":int})
df.iloc[:,list(range(0,9))]=df.iloc[:,list(range(0,9))].astype(int)
df[['p', 'q']] = df[['p', 'q']].astype(int)

# Retorna un arreglo de arreglos de números p,q representando los multipletes existentes
multiplets = df[['p','q']].drop_duplicates().values
df

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,3,0,1,2,1,1,2,0,1,-sqrt(1/3),3,0,1,1
304,3,0,1,2,1,2,2,0,0,-sqrt(1/6),3,0,1,1
305,3,0,2,2,1,1,2,0,2,-sqrt(1/6),3,0,1,1
306,3,0,2,2,1,2,2,0,1,-sqrt(1/3),3,0,1,1


In [18]:
multiplets

array([[0, 0],
       [0, 3],
       [1, 1],
       [2, 2],
       [3, 0]])

In [19]:
from sympy import *
# Calcula el cgc para un caso específico dentro de un df previamente filtrado
def cgc_md(initial1, initial2, final, ddf):
    dff = ddf.loc[(df['k1']==initial1[0])&(df['l1']==initial1[1])&(df['m1']==initial1[2])]
    dff = dff.loc[(df['k2']==initial2[0])&(df['l2']==initial2[1])&(df['m2']==initial2[2])]
    dff = dff.loc[(df['k']==final[0])&(df['l']==final[1])&(df['m']==final[2])]

    cgc_str=dff.loc[:,'cg_coef'].values[0]
    cgc = parse_expr(cgc_str, evaluate=0)
    return cgc

# Verifica si el df filtrado es simétrico o antisimétrico
def is_symmetric(sdf):
    p1 = sdf[['k1','l1','m1']].head(1).values[0]
    p2 = sdf[['k2','l2', 'm2']].head(1).values[0]
    pf = sdf[['k', 'l', 'm']].head(1).values[0]
    val1 = cgc_md(p1, p2, pf, sdf)
    val2 = cgc_md(p2, p1, pf, sdf)
    if val1 == val2:
        return 'S'
    else:
        return 'A'

In [20]:
Symmetry = []
Multiplets = [] 
Degeneracy = []
Dtot = []
Index = []
for i in range(len(multiplets)):
    # Retorna un df del multiplete indicado en la función
    m = df.loc[(df['p']==multiplets[i,0])&(df['q']==multiplets[i,1])]
    
    # Retorna una lista con las degeneraciones del multiplete m
    degeneracy = m['degeneracy'].value_counts().index.tolist()
    
    # Recolecta los valores de dtot en cada multiplete
    dtot = m['dtot'].value_counts().index.tolist()
    
    if len(degeneracy)>1:
        for j in range(len(degeneracy)):
            Multiplets.append([multiplets[i,0], multiplets[i,1]])
            
            # Divide el multiplete en df por degeneraciones
            d = m.loc[df['degeneracy']==degeneracy[j]]
            
            Index.append(d.index)
            Symmetry.append(is_symmetric(d))
            Degeneracy.append(degeneracy[j])
            # Repite los valores de dtot por si hay degeneración
            Dtot.append(dtot[0])

    else:
        Index.append(m.index)
        Multiplets.append([multiplets[i,0], multiplets[i,1]])
        Symmetry.append(is_symmetric(m))
        Degeneracy.append(degeneracy[0])
        Dtot.append(dtot[0])

# Elabora un df para describir cada multiplete
df_describe = pd.DataFrame()
MultipletsA = np.array(Multiplets)
df_describe['p'] = MultipletsA[:,0]
df_describe['q'] = MultipletsA[:,1]
df_describe['degeneracy'] = Degeneracy
df_describe['dtot'] = Dtot
df_describe['is_symm'] = Symmetry
df_describe

,p,q,degeneracy,dtot,is_symm
0,0,0,1,1,S
1,0,3,1,1,A
2,1,1,2,2,S
3,1,1,1,2,A
4,2,2,1,1,S
5,3,0,1,1,A


In [21]:
# Agrega una columna extra al df original
extra =[]
for i in range(df.shape[0]):
    extra.append('x')
df.loc[:,'is_symm']=extra

# Asigna correctamente la característica si es simétrico o antisimétrico
for i in range(len(Index)):
    for j in Index[i]:
        df.at[Index[i],'is_symm']=df_describe.loc[i, 'is_symm']
df

,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
0,0,0,0,1,0,0,2,1,2,sqrt(1/8),0,0,1,1,S
1,0,0,0,1,0,1,2,1,1,-sqrt(1/8),0,0,1,1,S
2,0,0,0,1,1,1,1,1,1,-sqrt(1/8),0,0,1,1,S
3,0,0,0,2,0,0,2,0,2,sqrt(1/8),0,0,1,1,S
4,0,0,0,2,0,1,2,0,1,-sqrt(1/8),0,0,1,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,3,0,1,2,1,1,2,0,1,-sqrt(1/3),3,0,1,1,A
304,3,0,1,2,1,2,2,0,0,-sqrt(1/6),3,0,1,1,A
305,3,0,2,2,1,1,2,0,2,-sqrt(1/6),3,0,1,1,A
306,3,0,2,2,1,2,2,0,1,-sqrt(1/3),3,0,1,1,A


### Algoritmo para el cálculo de coeficientes

#### Abrebiaciones
- I Isoespín
- Y Hipercarga
- I3 Tercera componente de Isoespín

## TO DO
1. Empezar a formar la matriz de coeficientes de decaimiento con el algoritmo
2. Dp_lambda_p = [cgc(klm, k1l1m1, k2l2m2, octete, S)]

Otra forma de buscar un valor dentro de una columna de un df es:
~~~
df[df['multiplet'].str.contains('00')]
~~~

In [22]:
e = Matrix([1,1]) # Dibarión inicial
e1 = Matrix([1,1]) # Barión final 1
e2 = Matrix([1,1]) # Barión final 2

In [23]:
dimension = (1+e[0])*(1+e[1])*(2+sum(e))/2
if e[0]<e[1]:
    print('dimension ', dimension, '*')
else:
    print('dimension ', dimension)
Imax = (e[0]+e[1])/2
print('Imax ', Imax)
Ymax = (e[0]-e[1])/3
print('Ymax ', Ymax)
Range = 2*Imax +1
print('Range ', Range)
if e1==e2:
    print('S and A')

dimension  8
Imax  1
Ymax  0
Range  3
S and A


In [24]:
# Retorna las degeneraciones de SU(2) dado un valor de I
def dec_su2(nmax):
    row = []
    
    rowImax = nmax
    size_m = 2*nmax + 1
    for i in range(size_m):
        row.insert(0,nmax)
        nmax = nmax -1
    return row

# Retorna el grupo de partículas pertenecientes al nivel solicitado
def level_particles(level):
    # Retorna la lista de la linea más larga en todo el multiplete
    I3max_line = dec_su2(Imax) # es una lista

    subIsospin3 = []
    subIsospin = []
    subHipercharge = []
    
    # Delimita los valores máximos y mínimos por nivel horizontal
    a = level-1
    b = len(I3max_line)-level + 1
    subIsospin3.append(I3max_line[a:b])
    Imax_line = I3max_line[b-1] # es un número
    list_subI = []
    list_subI.append(Imax_line)
    list_subI = list_subI*(b-a)
    subIsospin.append(list_subI)
    subImax = I3max_line[-level]
    list_subY=[]
    list_subY.append(Ymax)
    list_subY = list_subY*(b-a)
    subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia abajo formando grupos de partículas
    prow = []
    Inew = subImax
    Yrow1 = []
    Ynew1 = Ymax
    
    for i in range(e[0]-level+1):
        Inew = nsimplify(Inew -1/2)
        prow.append(Inew)
        Ynew1 = nsimplify(Ynew1 - 1)
        Yrow1.append(Ynew1)
    
    for i in range(len(prow)):
        subIsospin3.append(dec_su2(prow[i]))
        
        len_p = len(I3max_line)-i-1
        list_subI = []
        list_subI.append(prow[i])
        list_subI = list_subI*len_p
        subIsospin.append(list_subI)
        
        list_subY=[]
        list_subY.append(Yrow1[i])
        list_subY = list_subY*len_p
        subHipercharge.append(list_subY)
    
    # Recorre sobre la línea vertical hacia arriba formando grupos de partículas
    qrow = []
    Inew2 = subImax
    Yrow2 = []
    Ynew2 = Ymax
    for i in range(e[1]-level+1):
        Inew2 = nsimplify(Inew2 -1/2)
        qrow.insert(i,Inew2)
        Ynew2 = nsimplify(Ynew2 + 1)
        Yrow2.append(Ynew2)
        
    for i in range(len(qrow)):
        
        subIsospin3.insert(0,dec_su2(qrow[i]))
        list_subI = []
        list_subI.append(qrow[i])
        len_q = len(I3max_line)-i-1
        list_subI = list_subI*len_q
        subIsospin.insert(0,list_subI)
        
        list_subY=[]
        list_subY.append(Yrow2[i])
        list_subY = list_subY*len_q
        subHipercharge.insert(0, list_subY)
        
    return subHipercharge, subIsospin, subIsospin3

# Retorna el número de partículas en el grupo del nivel solicitado
def count_particles(levelp):
    tot_particles = 0
    for i in range(len(levelp)):
        tot_particles = tot_particles + len(levelp[i])
    return tot_particles

In [25]:
I3 = []
I = []
Y = []
npart = 0
for i in range(dimension):
    level = i+1
    subY, subI, subI3 = level_particles(level)
    npart = npart + count_particles(subI3)
    I3.append(subI3)
    I.append(subI)
    Y.append(subY)
    
    if npart == dimension:
        break

In [26]:
mB = []
for i in range(len(Y)):
    for j in range(len(Y[i])):
        for k in range(len(Y[i][j])):
            mB.append([Y[i][j][k], I[i][j][k], I3[i][j][k]])
print(mB)

[[1, 1/2, -1/2], [1, 1/2, 1/2], [0, 1, -1], [0, 1, 0], [0, 1, 1], [-1, 1/2, -1/2], [-1, 1/2, 1/2], [0, 0, 0]]


In [27]:
# Transforma los números cuánticos de una particula de y, i, iz al tipo k, l, m
def transform(particle):
    # Se escoge el multiplete con que se va a trabajar
    p = e[0]
    q = e[1]
    
    # Convierte enteros k, l, m
    k = int((p+2*q)/3 + particle[0]/2 + particle[1])
    l = int((p+2*q)/3 + particle[0]/2 - particle[1])
    m = int((p+2*q)/3 + particle[0]/2 + particle[2])
    return k,l,m

# Recibe una lista de lista de numeros y, i, iz la cambia a lista de enteros k,l,m
def transform_m(listm):
    list_klm = []
    for i in range(len(listm)):
        list_klm.append(transform(listm[i]))
    array_klm = np.array(list_klm)
    return array_klm

In [28]:
# Se transforman los números Y, I, I3 a k, l, m de todas las partículas de los multipletes
e0klm = transform_m(mB)
e1klm = transform_m(mB)
e2klm = transform_m(mB)

In [29]:
e0klm

array([[2, 1, 1],
       [2, 1, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1]])

In [30]:
Gm = Matrix([])
count = 0
for i in range(len(e0klm)):
    dfLS=df.loc[(df['is_symm']=='S')]
    dfLS=dfLS.loc[(df['p']==e[0])&(df['q']==e[1])]
    dfLS=dfLS.loc[(df['k']==e0klm[i,0])&(df['l']==e0klm[i,1])&(df['m']==e0klm[i,2])]
    print(dfLS)
    for j in range(dfLS.shape[0]):
        D=list(dfLS.iloc[j,0:3].values)
        B1=list(dfLS.iloc[j,3:6].values)
        B2=list(dfLS.iloc[j,6:9].values)
        cgci = cgc_md(B1,B2,D,dfLS)
        tag = [D, B1, B2]
        Gi = 'G'+str(count)
        count = count + 1
        Gm = Gm.row_insert(count, Matrix([[cgci,tag, Gi]]))

     k  l  m  k1  l1  m1  k2  l2  m2      cg_coef  p  q  degeneracy  dtot  \
153  2  1  1   1   1   1   2   1   1  -sqrt(1/20)  1  1           2     2   
155  2  1  1   2   0   0   2   1   2   sqrt(3/10)  1  1           2     2   
156  2  1  1   2   0   1   2   1   1  -sqrt(3/20)  1  1           2     2   
159  2  1  1   2   1   1   1   1   1  -sqrt(1/20)  1  1           2     2   
161  2  1  1   2   1   1   2   0   1  -sqrt(3/20)  1  1           2     2   
162  2  1  1   2   1   2   2   0   0   sqrt(3/10)  1  1           2     2   

    is_symm  
153       S  
155       S  
156       S  
159       S  
161       S  
162       S  
     k  l  m  k1  l1  m1  k2  l2  m2      cg_coef  p  q  degeneracy  dtot  \
154  2  1  2   1   1   1   2   1   2  -sqrt(1/20)  1  1           2     2   
157  2  1  2   2   0   1   2   1   2   sqrt(3/20)  1  1           2     2   
158  2  1  2   2   0   2   2   1   1  -sqrt(3/10)  1  1           2     2   
160  2  1  2   2   1   2   1   1   1  -sqrt(1/20)  1  

In [54]:
Gm[0,1]

[[2, 1, 1], [1, 1, 1], [2, 1, 1]]

In [32]:
count

53

In [33]:
Gm[0:24,:]

Matrix([
[ -sqrt(5)/10, [[2, 1, 1], [1, 1, 1], [2, 1, 1]],  G0],
[ sqrt(30)/10, [[2, 1, 1], [2, 0, 0], [2, 1, 2]],  G1],
[-sqrt(15)/10, [[2, 1, 1], [2, 0, 1], [2, 1, 1]],  G2],
[ -sqrt(5)/10, [[2, 1, 1], [2, 1, 1], [1, 1, 1]],  G3],
[-sqrt(15)/10, [[2, 1, 1], [2, 1, 1], [2, 0, 1]],  G4],
[ sqrt(30)/10, [[2, 1, 1], [2, 1, 2], [2, 0, 0]],  G5],
[ -sqrt(5)/10, [[2, 1, 2], [1, 1, 1], [2, 1, 2]],  G6],
[ sqrt(15)/10, [[2, 1, 2], [2, 0, 1], [2, 1, 2]],  G7],
[-sqrt(30)/10, [[2, 1, 2], [2, 0, 2], [2, 1, 1]],  G8],
[ -sqrt(5)/10, [[2, 1, 2], [2, 1, 2], [1, 1, 1]],  G9],
[-sqrt(30)/10, [[2, 1, 2], [2, 1, 1], [2, 0, 2]], G10],
[ sqrt(15)/10, [[2, 1, 2], [2, 1, 2], [2, 0, 1]], G11],
[-sqrt(30)/10, [[2, 0, 0], [1, 0, 0], [2, 1, 1]], G12],
[   sqrt(5)/5, [[2, 0, 0], [1, 1, 1], [2, 0, 0]], G13],
[   sqrt(5)/5, [[2, 0, 0], [2, 0, 0], [1, 1, 1]], G14],
[           0, [[2, 0, 0], [2, 0, 0], [2, 0, 1]], G15],
[           0, [[2, 0, 0], [2, 0, 1], [2, 0, 0]], G16],
[-sqrt(30)/10, [[2, 0, 0], [2, 1, 1], [

In [34]:
Gm[24:53,:]

Matrix([
[           0, [[2, 0, 1], [2, 0, 2], [2, 0, 0]], G24],
[-sqrt(15)/10, [[2, 0, 1], [2, 1, 1], [1, 0, 1]], G25],
[-sqrt(15)/10, [[2, 0, 1], [2, 1, 2], [1, 0, 0]], G26],
[-sqrt(30)/10, [[2, 0, 2], [1, 0, 1], [2, 1, 2]], G27],
[   sqrt(5)/5, [[2, 0, 2], [1, 1, 1], [2, 0, 2]], G28],
[   sqrt(5)/5, [[2, 0, 2], [2, 0, 2], [1, 1, 1]], G29],
[           0, [[2, 0, 2], [2, 0, 1], [2, 0, 2]], G30],
[           0, [[2, 0, 2], [2, 0, 2], [2, 0, 1]], G31],
[-sqrt(30)/10, [[2, 0, 2], [2, 1, 2], [1, 0, 1]], G32],
[ -sqrt(5)/10, [[1, 0, 0], [1, 0, 0], [1, 1, 1]], G33],
[ sqrt(15)/10, [[1, 0, 0], [1, 0, 0], [2, 0, 1]], G34],
[-sqrt(30)/10, [[1, 0, 0], [1, 0, 1], [2, 0, 0]], G35],
[ -sqrt(5)/10, [[1, 0, 0], [1, 1, 1], [1, 0, 0]], G36],
[-sqrt(30)/10, [[1, 0, 0], [2, 0, 0], [1, 0, 1]], G37],
[ sqrt(15)/10, [[1, 0, 0], [2, 0, 1], [1, 0, 0]], G38],
[ -sqrt(5)/10, [[1, 0, 1], [1, 0, 1], [1, 1, 1]], G39],
[ sqrt(30)/10, [[1, 0, 1], [1, 0, 0], [2, 0, 2]], G40],
[-sqrt(15)/10, [[1, 0, 1], [1, 0, 1], [

In [35]:
Gm.shape

(53, 3)

In [36]:
transform([1, 1/2, 1/2])

(2, 1, 2)

In [37]:
def transformpq(particle, epq):
    # Se escoge el multiplete con que se va a trabajar
    p = epq[0]
    q = epq[1]
    
    # Convierte enteros k, l, m
    k = int((p+2*q)/3 + particle[0]/2 + particle[1])
    l = int((p+2*q)/3 + particle[0]/2 - particle[1])
    m = int((p+2*q)/3 + particle[0]/2 + particle[2])
    return k,l,m

In [38]:
#i=0
for i in range(len(mB)):
    print(transformpq(mB[i], [0,0]))

(1, 0, 0)
(1, 0, 1)
(1, -1, -1)
(1, -1, 0)
(1, -1, 1)
(0, -1, -1)
(0, -1, 0)
(0, 0, 0)


In [39]:
df_describe

,p,q,degeneracy,dtot,is_symm
0,0,0,1,1,S
1,0,3,1,1,A
2,1,1,2,2,S
3,1,1,1,2,A
4,2,2,1,1,S
5,3,0,1,1,A


In [40]:
# Convierte df_desribe en un diccionario y birra la columna dtot
dictini = df_describe.to_dict('list')
dictfinal = df_describe.to_dict('list')
del dictini['dtot']
del dictfinal['dtot']

# Forma el df Inicial
for i in range(df_describe.shape[0]):
    if df_describe.iloc[i,3]>1:
        for j in range(df_describe.iloc[i,3]-1):
            dictini['p'].insert(i+(df_describe.iloc[i,3]-1)*(j+1), df_describe.iloc[i,0])
            dictini['q'].insert(i+(df_describe.iloc[i,3]-1)*(j+1), df_describe.iloc[i,1])
            dictini['degeneracy'].insert(i, df_describe.iloc[i,2])
            dictini['is_symm'].insert(i+(df_describe.iloc[i,3]-1)*(j+1), df_describe.iloc[i,4])

dfIni = pd.DataFrame(dictini)

# Forma el df Final
for i in range(df_describe.shape[0]):
    if df_describe.iloc[i,3]>1:
        for j in range(df_describe.iloc[i,3]-1):
            dictfinal['p'].insert(i, df_describe.iloc[i,0])
            dictfinal['q'].insert(i, df_describe.iloc[i,1])
            dictfinal['degeneracy'].insert(i+(df_describe.iloc[i,3]-1)*(j+1), df_describe.iloc[i,2])
            dictfinal['is_symm'].insert(i+(df_describe.iloc[i,3]-1)*(j+1), df_describe.iloc[i,4])

dfFinal = pd.DataFrame(dictfinal)

In [41]:
dfIni

,p,q,degeneracy,is_symm
0,0,0,1,S
1,0,3,1,A
2,1,1,2,S
3,1,1,1,S
4,1,1,2,A
5,1,1,1,A
6,2,2,1,S
7,3,0,1,A


In [42]:
dfFinal

,p,q,degeneracy,is_symm
0,0,0,1,S
1,0,3,1,A
2,1,1,2,S
3,1,1,2,S
4,1,1,1,A
5,1,1,1,A
6,2,2,1,S
7,3,0,1,A


In [43]:
# Separa cada df en sus partes simétricas y antisimétricas
dfIS=dfIni.loc[(dfIni['is_symm']=='S')]
dfIA=dfIni.loc[(dfIni['is_symm']=='A')]
dfFS=dfFinal.loc[(dfFinal['is_symm']=='S')]
dfFA=dfFinal.loc[(dfFinal['is_symm']=='A')]
dfIS

,p,q,degeneracy,is_symm
0,0,0,1,S
2,1,1,2,S
3,1,1,1,S
6,2,2,1,S


In [44]:
dfFS

,p,q,degeneracy,is_symm
0,0,0,1,S
2,1,1,2,S
3,1,1,2,S
6,2,2,1,S


In [45]:
# Considerando el valor de multiplets se calculan k, l, m dependiendo de valores p y q de un df dado
def Di_pq(Dklm, df):
    Dpq_l=[]
    for j in range(df.shape[0]):
        Dpq_l.append(transformpq(Dklm, df[['p','q']].values[j]))
    Dpq = np.array(Dpq_l)
    return Dpq

In [46]:
mB

[[1, 1/2, -1/2],
 [1, 1/2, 1/2],
 [0, 1, -1],
 [0, 1, 0],
 [0, 1, 1],
 [-1, 1/2, -1/2],
 [-1, 1/2, 1/2],
 [0, 0, 0]]

In [51]:
i=1
print(mB[i])
Di_pq(mB[i], dfIS)

[1, 1/2, 1/2]


array([[1, 0, 1],
       [2, 1, 2],
       [2, 1, 2],
       [3, 2, 3]])

In [48]:
i=0
print(mB[i])
Di_pq(mB[i], dfFS)

[1, 1/2, -1/2]


array([[1, 0, 0],
       [2, 1, 1],
       [2, 1, 1],
       [3, 2, 2]])

In [49]:
# Barre sobre cada Dibarión
i=0
# Barre sobre cada multiplete de un df dado
j=1
df11=df.loc[(df['k']==Di_pq(mB[i], dfIS)[j,0])&(df['l']==Di_pq(mB[i], dfIS)[j,1])&(df['m']==Di_pq(mB[i], dfIS)[j,2])]
df11=df11.loc[(df['p']==dfIS.iloc[j,0])&(df['q']==dfIS.iloc[j,1])]
df11=df11.loc[df['degeneracy']==dfIS.iloc[j,2]]
df11=df11.loc[(df['k1']==1)&(df['l1']==1)&(df['m1']==1)]
print(mB[i], Di_pq(mB[i], dfIS))
df11

[1, 1/2, -1/2] [[1 0 0]
 [2 1 1]
 [2 1 1]
 [3 2 2]]


,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
153,2,1,1,1,1,1,2,1,1,-sqrt(1/20),1,1,2,2,S


In [50]:
# Barre sobre cada Dibarión
i=0
# Barre sobre cada multiplete de un df dado
j=1
df22=df.loc[(df['k']==Di_pq(mB[i], dfFS)[j,0])&(df['l']==Di_pq(mB[i], dfFS)[j,1])&(df['m']==Di_pq(mB[i], dfFS)[j,2])]
df22=df22.loc[(df['p']==dfFS.iloc[j,0])&(df['q']==dfFS.iloc[j,1])]
df22=df22.loc[df['degeneracy']==dfFS.iloc[j,2]]
print(mB[i], Di_pq(mB[i], dfFS))
df22

[1, 1/2, -1/2] [[1 0 0]
 [2 1 1]
 [2 1 1]
 [3 2 2]]


,k,l,m,k1,l1,m1,k2,l2,m2,cg_coef,p,q,degeneracy,dtot,is_symm
153,2,1,1,1,1,1,2,1,1,-sqrt(1/20),1,1,2,2,S
155,2,1,1,2,0,0,2,1,2,sqrt(3/10),1,1,2,2,S
156,2,1,1,2,0,1,2,1,1,-sqrt(3/20),1,1,2,2,S
159,2,1,1,2,1,1,1,1,1,-sqrt(1/20),1,1,2,2,S
161,2,1,1,2,1,1,2,0,1,-sqrt(3/20),1,1,2,2,S
162,2,1,1,2,1,2,2,0,0,sqrt(3/10),1,1,2,2,S
